In [ ]:
### CBC SCRAPING CODE
### Authors: JONATHAN CHAN and PANDRAMISHI NAGA SIRISHA

###MOST RECENT UPDATE:  
##2020 MAY 15, 11:52AM
#wrote separate functions for each JSON element to be collected
#extract_json_items() will run for all articles, and will return null if not in proper format

#TO DO:
#write code for collecting JSON items from all articles returned in CBC API (code cell 4)
#write code to iterate through multiple pages of API call 
#done
#write code to store JSON in a JSON or text document
#Final run: collect 50-100 articles for each of 6 indicators

In [1]:
import urllib.request
import json 
from bs4 import BeautifulSoup
#from datetime import date
import requests
import json
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from datetime import datetime, timedelta
import datetime
import pytz
import dateutil.parser


In [2]:
#https://www.cbc.ca/search_api/v1/search?q=mortgage%20rate&sortOrder=relevance&page=100&fields=feed
def get_initial_url(search_term):
    """returns the URL of the first page API call given a search string"""
    
    words = search_term.split()
    url_prefix = "https://www.cbc.ca/search_api/v1/search?"
    query = "q=" + "%20".join(words)
    url_suffix = "&sortOrder=relevance&page=1&fields=feed"
    first_url = url_prefix + query + url_suffix
    #print("FIRST URL API CALL: ", first_url)
    return first_url
    
get_initial_url("interest rate index")

'https://www.cbc.ca/search_api/v1/search?q=interest%20rate%20index&sortOrder=relevance&page=1&fields=feed'

In [16]:
# yourdate = dateutil.parser.parse('2020-05-13T14:48:05.000Z')
# print(yourdate)
# yourdate > past

2020-05-13 14:48:05+00:00


NameError: name 'past' is not defined

In [60]:
#MODIFIED SCRAPE URL FUNCTION 

def scrape_urls(initial_url):
    """This function takes in the first query url and scrapes all other articles from past 1 year and returns 
    the urls of such articles"""
    count = 0
    url_list = []
    current_date = datetime.datetime.now(datetime.timezone.utc)
    last_year_date = current_date - timedelta(days=365)
    main_url = initial_url
    r = requests.get(initial_url)
    info = r.json()
     
    for json_dict in info:
        if dateutil.parser.parse(json_dict['publishtime']) > last_year_date:
            url_list.append(json_dict["url"])
            
    page_num = 2     
    while len(info) != 0:
        if page_num >= 1000: #internal server error after page 1000
            break
            
        if page_num % 20 == 0:
            print("API CALLS SCRAPED: ", str(page_num))
            print("ARTICLES IN URL LIST: ", len(url_list))
            
        split_url = main_url.split("page")
        new_url = split_url[0] + "page=" + str(page_num) + "&fields=feed" 
        r = requests.get(new_url)
        info = r.json()
        
        for json_dict in info:
            if dateutil.parser.parse(json_dict['publishtime']) > last_year_date:
                url_list.append(json_dict["url"])
        page_num += 1
        
    print("FINAL URLS FROM THE PAST YEAR: ", len(url_list))
    
    return url_list
    

API CALLS SCRAPED:  20
ARTICLES IN URL LIST:  83
API CALLS SCRAPED:  40
ARTICLES IN URL LIST:  148
API CALLS SCRAPED:  60
ARTICLES IN URL LIST:  186
API CALLS SCRAPED:  80
ARTICLES IN URL LIST:  217
API CALLS SCRAPED:  100
ARTICLES IN URL LIST:  244
API CALLS SCRAPED:  120
ARTICLES IN URL LIST:  256
API CALLS SCRAPED:  140
ARTICLES IN URL LIST:  299
API CALLS SCRAPED:  160
ARTICLES IN URL LIST:  310
API CALLS SCRAPED:  180
ARTICLES IN URL LIST:  314
API CALLS SCRAPED:  200
ARTICLES IN URL LIST:  317
API CALLS SCRAPED:  220
ARTICLES IN URL LIST:  325
LEN OF URLS FOR THIS YEAR:  325


In [4]:
#CODE WRITTEN BY SIRISHA

# def scrape_urls(url):
#     """This function takes in the first query url and scrapes all other articles from past 1 year and returns 
#     the urls of such articles"""
#     count = 0
#     url_list = []
#     current_date = datetime.datetime.now(datetime.timezone.utc)
#     last_year_date = current_date - timedelta(days=365)
#     main_url = url
#     r = requests.get(url)
#     info = r.json()
#     last_retrieved_items_count= len(info)
    
#     for i in info:
#         if dateutil.parser.parse(i['publishtime']) > last_year_date:
#             url_list.append(i['url'])
#             count += 1
        
#     page_number = 2
    
#     while last_retrieved_items_count > 0 :
# #    while count < 250 :
#         split_url = main_url.split('page')
#         new_url = split_url[0] + "page=" + str(page_number) + "&fields=feed" 
#         r = requests.get(new_url)
#         info = r.json()
#         last_retrieved_items_count= len(info)
        
#         for i in info:
            
#             if dateutil.parser.parse(i['publishtime']) > last_year_date:
#                 url_list.append(i['url'])
#                 count += 1
                
#         page_number += 1   
#         print(page_number)
#     return url_list

# # first_url = get_initial_url("supernova")
# # all_urls = scrape_urls(first_url)



In [5]:
def get_author(soup):
    """returns the author of a BeautifulSoup article if it exists, None if cannot be found
    
    Assume author info is contained within span tag (class: authorText)
    """
    author_span = soup.find("span", {"class": "authorText"})
    
    if author_span:
        return author_span.text
    else:
        #print("No author found in article!")
        return None

In [6]:
def get_title(soup):
    """returns the title of a BeautifulSoup article if it exists, None if cannot be found
    
    Assume title info is contained within h1 tag (class: detailHeadline)
    """
    title_tag = soup.find("h1", {"class": "detailHeadline"})
    
    if title_tag:
        title_text = title_tag.text
        return title_text
    else:
        #print("no title found in article!")
        return None

In [7]:
def get_desc(soup):
    """returns the description of a BeautifulSoup article if it exists, None if not
    
    Assume description is contained within h2 tag (class: deck)
    """
    desc_tag = soup.find("h2", {"class": "deck"})
    
    if desc_tag:
        desc_text = desc_tag.text
        return desc_text
    else:
        #print("No description found in article!")
        return None
    

In [8]:
def get_url_to_image(soup):
    """returns the url to the header image of a CBC article (BeautifulSoup) if it exists, None if not
    
    Assume image url is contained within src attribute of img tag 
    """
    main_image_tag = soup.find("figure", {"class": "imageMedia leadmedia-story full"})
    
    if main_image_tag:
        main_image_url = main_image_tag.find("img").attrs["src"]
        return main_image_url
    else:
        #print("No main header image found in article!")
        return None
        

In [9]:
def get_publish_time(soup):
    """returns a tuple of publish time string and datetime string if found in article, None if not
    
    Assume time is contained within time tag (class: timestamp)
    """
    time_tag = soup.find("time", {"class": "timeStamp"})
    if time_tag:
        datetime_str = time_tag.attrs["datetime"]
        
        #NOTE: if we want to return a datetime object, error when writing to JSON
        #datetime_obj = parser.isoparse(datetime_str)
        #SOLUTION: return as string for now, convert to datetime object later in pipeline
        
        #format of time_tag.text: 
        timetext_str = time_tag.text.split("|")[0].replace("Posted: ", "").strip()
        return (timetext_str, datetime_str)
    else:
        #print("No time information found in article!")
        return None


In [10]:
def get_source(soup, specify_source_type=True):
    """Returns the source of the article if it exists
    if specify_source_type, subdivision of CBC will be returned
    if not, "CBC" will be returned as the source
    
    
    Assume that source always starts with "CBC" (Ex: "CBC news", "CBC radio")
    Assume that source comes before span tag (class: bullet)
    """
    
    #source appears before <span class="bullet"> · </span>
    #if author is attached, there are two bullet tags
    #if no author attached, there is one bullet tag
    source = None
    
    if specify_source_type:
        bullet_spans = soup.find_all("span", {"class": "bullet"})
        for bullet_span in bullet_spans:
            previous_str = str(bullet_span.previous_sibling)
            if previous_str.startswith("CBC"):
                source = previous_str
    else:
        
        source = "CBC"
    
    if source:
        return source
    else:
        #print("no source found in article!")
        return None
    

In [11]:

def get_content(soup, as_string=True):
    """Returns the text content from a CBC article (as BeautifulSoup object)
    if as_string is True, return content as one string,
    if as_string is False, return content as list of paragraph strings
    
    Input: BeautifulSoup object, boolean
    
    """
    
    story_tag = soup.find("div", {"class": "story"}) 
    content_list = []
    
    if story_tag:
        for p_tag in story_tag.find_all("p"):
            p_text = p_tag.text + "\n"
            content_list.append(p_text)

        if as_string:
            final_content = "".join(content_list)
        else:
            final_content = content_list #return content as list of paragraph strings

        return final_content
    else:
        #print("no content found in article!")
        return None
    
# get_content(soup)

In [47]:

def extract_json_items(url, specify_source_type=True):
    """Returns a json containing the following items from a CBC article:
        url: the url of the article
        urlToImage: the url of the header image
        title: the title of the article 
        description: subheader of the article
        author: author (note that some articles do not specify author)
        source: CBC if specify_source_type == False, subdivision of CBC if True (ex: "CBC radio")
        publishedAt: tuple of (date_string, datetime object)
        
        input: url returned from CBC API in "url" field (missing "http:" as part of URL)
    """
    json_dict = {}
#     output_list = [] 
    article_url = "http:" + url
    
    #get HTML from article URL into BeautifulSoup
    try:
        html_bytes = urllib.request.urlopen(article_url)
  
    except HTTPError as e:
        print('Error code: ', e.code)
        return None
    except URLError as e:
        print('Reason: ', e.reason)
        return None

    else:    
        mybytes = html_bytes.read()
        html = mybytes.decode("utf8")
        html_bytes.close()
        soup = BeautifulSoup(html, 'html.parser')
        author_name = get_author(soup)
        title_text = get_title(soup)
        desc_text = get_desc(soup)
        image_url = get_url_to_image(soup)
        publish_time = get_publish_time(soup)
        news_source = get_source(soup)
        content = get_content(soup, True)
        
        json_dict["author"] = author_name
        json_dict["title"] = title_text 
        json_dict["description"] = desc_text
        json_dict["url"] = article_url
        json_dict["urlToImage"] = image_url
        json_dict["publishedAt"] = publish_time
        json_dict["source"] = news_source
        json_dict["content"] = content
#         print("JSON KEYS AND VALUES: ")
#         for key, item in json_dict.items():
#             print(key + ": " + str(item))
#         print("-----")
        
        final_json = json.dumps(json_dict)
        return json_dict


In [63]:
#MAIN FUNCTION CREATED BY SIRISHA
def main(query):
    """"""
    first_url = get_initial_url(query)
    all_urls = scrape_urls(first_url)
    json_list = []
    json_count = 0
    for each_url in all_urls:
        retrieved_json  = extract_json_items(each_url)
        if retrieved_json is not None:
            #print(each_url)
            json_list.append(retrieved_json)
            
            json_count += 1
            if json_count % 20 == 0:
                print("JSONS ADDED:", len(json_count))
    
    full_query = query.split(" ")
    file_name_prefix = "_".join(full_query)
    print(file_name_prefix)
    
    with open( file_name_prefix + '_' +'CBC_article' + '.json', 'w') as json_file:
        json.dump(json_list, json_file)
    
    return json_list



## Mortgage Rates

In [ ]:
# cbc_mr_article = main("mortgage rates")
# print(len(cbc_mr_article))
# print(cbc_mr_article[0])

## Interest rates

In [ ]:
# cbc_ir_article =  main("interest rate")
# print(len(cbc_hp_article))
# print(cbc_hp_article[0])

## Housing price

In [74]:
# cbc_hp_article = main('housing price')
# print(len(cbc_hp_article))
# print(cbc_hp_article[0])

## Employment

In [75]:
cbc_e_article = main('employment')
print(len(cbc_e_article))
print(cbc_e_article[0])

API CALLS SCRAPED:  20
ARTICLES IN URL LIST:  101
API CALLS SCRAPED:  40
ARTICLES IN URL LIST:  110
API CALLS SCRAPED:  60
ARTICLES IN URL LIST:  239
API CALLS SCRAPED:  80
ARTICLES IN URL LIST:  438
API CALLS SCRAPED:  100
ARTICLES IN URL LIST:  617
API CALLS SCRAPED:  120
ARTICLES IN URL LIST:  765


KeyboardInterrupt: 

In [76]:
initial_e_url = get_initial_url("employment")
all_e_urls = scrape_urls(initial_e_url)

API CALLS SCRAPED:  20
ARTICLES IN URL LIST:  102
API CALLS SCRAPED:  40
ARTICLES IN URL LIST:  112
API CALLS SCRAPED:  60
ARTICLES IN URL LIST:  242
API CALLS SCRAPED:  80
ARTICLES IN URL LIST:  441
API CALLS SCRAPED:  100
ARTICLES IN URL LIST:  618
API CALLS SCRAPED:  120
ARTICLES IN URL LIST:  767
API CALLS SCRAPED:  140
ARTICLES IN URL LIST:  877
API CALLS SCRAPED:  160
ARTICLES IN URL LIST:  975
API CALLS SCRAPED:  180
ARTICLES IN URL LIST:  1058
API CALLS SCRAPED:  200
ARTICLES IN URL LIST:  1120
API CALLS SCRAPED:  220
ARTICLES IN URL LIST:  1181
API CALLS SCRAPED:  240
ARTICLES IN URL LIST:  1232
API CALLS SCRAPED:  260
ARTICLES IN URL LIST:  1288
API CALLS SCRAPED:  280
ARTICLES IN URL LIST:  1344
API CALLS SCRAPED:  300
ARTICLES IN URL LIST:  1389
API CALLS SCRAPED:  320
ARTICLES IN URL LIST:  1437
API CALLS SCRAPED:  340
ARTICLES IN URL LIST:  1470
API CALLS SCRAPED:  360
ARTICLES IN URL LIST:  1516
API CALLS SCRAPED:  380
ARTICLES IN URL LIST:  1548
API CALLS SCRAPED:  400


In [79]:
json_list = []
json_count = 0
for each_url in all_e_urls:
    retrieved_json  = extract_json_items(each_url)
    if retrieved_json is not None:
        #print(each_url)
        json_list.append(retrieved_json)

        json_count += 1
        if json_count % 20 == 0:
            print("JSONS ADDED:", json_count)

JSONS ADDED: 20
JSONS ADDED: 40
Error code:  404
JSONS ADDED: 60
JSONS ADDED: 80
JSONS ADDED: 100
JSONS ADDED: 120
JSONS ADDED: 140
JSONS ADDED: 160
JSONS ADDED: 180
JSONS ADDED: 200
JSONS ADDED: 220
JSONS ADDED: 240
JSONS ADDED: 260
JSONS ADDED: 280
JSONS ADDED: 300
JSONS ADDED: 320
JSONS ADDED: 340
JSONS ADDED: 360
JSONS ADDED: 380
JSONS ADDED: 400
JSONS ADDED: 420
JSONS ADDED: 440
JSONS ADDED: 460
JSONS ADDED: 480
JSONS ADDED: 500
JSONS ADDED: 520
JSONS ADDED: 540
JSONS ADDED: 560
JSONS ADDED: 580
JSONS ADDED: 600
JSONS ADDED: 620
JSONS ADDED: 640
JSONS ADDED: 660
JSONS ADDED: 680
JSONS ADDED: 700
JSONS ADDED: 720
JSONS ADDED: 740
JSONS ADDED: 760
JSONS ADDED: 780
JSONS ADDED: 800
JSONS ADDED: 820
JSONS ADDED: 840
Error code:  404
JSONS ADDED: 860
JSONS ADDED: 880
JSONS ADDED: 900
Error code:  500
JSONS ADDED: 920
JSONS ADDED: 940
JSONS ADDED: 960
JSONS ADDED: 980
JSONS ADDED: 1000
Error code:  500
JSONS ADDED: 1020
JSONS ADDED: 1040
JSONS ADDED: 1060
JSONS ADDED: 1080
JSONS ADDED: 

ConnectionResetError: [Errno 54] Connection reset by peer

In [80]:
query = "employment"
full_query = query.split(" ")
file_name_prefix = "_".join(full_query)
print(file_name_prefix)

with open( file_name_prefix + '_' +'CBC_article' + '.json', 'w') as json_file:
    json.dump(json_list, json_file)

employment


In [ ]:
# ind = 0
# for json_dict in cbc_e_article:
#     if json_dict["title"]:
#         #print(json_dict)
#         print(ind)
#         ind += 1


## GDP

In [61]:
# cbc_gdp_article = main('GDP')
# print(len(cbc_gdp_article))
# print(cbc_gdp_article[0])

API CALLS SCRAPED:  20
ARTICLES IN URL LIST:  86
API CALLS SCRAPED:  40
ARTICLES IN URL LIST:  150
API CALLS SCRAPED:  60
ARTICLES IN URL LIST:  190
API CALLS SCRAPED:  80
ARTICLES IN URL LIST:  222
API CALLS SCRAPED:  100
ARTICLES IN URL LIST:  252
API CALLS SCRAPED:  120
ARTICLES IN URL LIST:  264
API CALLS SCRAPED:  140
ARTICLES IN URL LIST:  301
API CALLS SCRAPED:  160
ARTICLES IN URL LIST:  310
API CALLS SCRAPED:  180
ARTICLES IN URL LIST:  314
API CALLS SCRAPED:  200
ARTICLES IN URL LIST:  318
API CALLS SCRAPED:  220
ARTICLES IN URL LIST:  326
LEN OF URLS FOR THIS YEAR:  326
//www.cbc.ca/news/business/february-gdp-statscan-1.5550479
//www.cbc.ca/news/business/january-gdp-1.5515908
//www.cbc.ca/player/play/2676409093
//www.cbc.ca/news/business/canada-gdp-november-1.5447173
//www.cbc.ca/radio/costofliving/the-cost-of-running-either-for-election-or-a-deficit-1.5323745/canada-s-debt-to-gdp-ratio-isn-t-a-major-election-issue-but-should-it-be-1.5327355
//www.cbc.ca/news/business/us-eco

//www.cbc.ca/news/canada/post-secondary-enrollment-1.5554348
//www.cbc.ca/news/business/stock-market-trump-uncertainty-1.5347006
//www.cbc.ca/news/canada/calgary/road-ahead-opinion-trevor-tombe-alberta-fiscal-reckoning-1.5546481
//www.cbc.ca/news/business/home-price-heloc-spending-1.5291558
//www.cbc.ca/news/canada/toronto/2020-fiscal-update-1.5510186
//www.cbc.ca/radio/day6/trump-s-odds-for-re-election-house-hippos-are-back-empire-strikes-door-bahamas-tourism-after-dorian-more-1.5306553/a-month-after-dorian-the-bahamas-still-needs-aid-but-officials-say-it-needs-tourists-even-more-1.5306557
//www.cbc.ca/news/canada/toronto/ontario-covid19-budget-spending-plan-analysis-1.5509955
//www.cbc.ca/radio/day6/a-month-after-dorian-the-bahamas-still-needs-aid-but-officials-say-it-needs-tourists-even-more-1.5309524
//www.cbc.ca/news/business/canada-gdp-banks-growth-1.5161849
//www.cbc.ca/news/canada/british-columbia/sonia-furstenau-announces-she-s-running-for-leader-of-b-c-green-party-1.5441601
/

//www.cbc.ca/news/canada/saskatchewan/women-entrepreneurship-strategy-announcement-1.5226056
//www.cbc.ca/news/world/xi-jinping-global-ambition-1.5373557
//www.cbc.ca/news/business/covid-coronavirus-economy-bailouts-pittis-1.5503811
//www.cbc.ca/news/entertainment/jamaica-central-bank-reggae-1.5180119
//www.cbc.ca/news/canada/montreal/quebec-minimum-wage-raise-1.5401147
//www.cbc.ca/news/canada/british-columbia/vancouver-needs-10-000-affordable-housing-units-a-year-to-address-rental-backlog-report-advises-1.5393571
//www.cbc.ca/news/canada/calgary/alberta-budget-2019-trevor-tombe-1.5337962
//www.cbc.ca/news/canada/prince-edward-island/pei-economy-job-losses-covid-coronavirus-1.5526397
//www.cbc.ca/news/business/north-american-markets-1.5375839
//www.cbc.ca/news/business/north-american-markets-1.5375839
//www.cbc.ca/news/business/north-american-markets-1.5375839
//www.cbc.ca/news/politics/carbon-pricing-tax-climate-change-emissions-1.5374481
//www.cbc.ca/news/canada/montreal/budget-queb

//www.cbc.ca/news/politics/cong-peiwu-china-ambassador-kovrig-spavor-1.5370503
//www.cbc.ca/news/business/teamsters-cdn-strike-1.5369359
//www.cbc.ca/news/canada/nova-scotia/budget-aimed-at-health-care-education-and-poverty-likely-to-fuel-n-s-election-talk-1.5475294
//www.cbc.ca/news/politics/canada-votes-newsletter-issue21-andrew-scheer-abortion-lgbt-rights-1.5266068
//www.cbc.ca/news/politics/ndp-platform-costing-fact-check-1.5318913
//www.cbc.ca/news/canada/calgary/alberta-trevor-tombe-recession-analysis-1.5299896
//www.cbc.ca/news/politics/trudeau-liberal-platform-release-election-1.5302042
//www.cbc.ca/news/politics/liberal-health-care-fact-check-1.5295449
//www.cbc.ca/news/opinion/negative-rates-1.5244661
//www.cbc.ca/news/technology/climate-change-mitigation-co-benefits-1.5205552
//www.cbc.ca/news/business/climate-change-moody-s-1.5199652
//www.cbc.ca/news/business/climate-change-moody-s-1.5199652
//www.cbc.ca/news/canada/coronavirus-covid19-april22-canada-world-1.5539330
//www.

## Stock Market

In [65]:
# cbc_tsx_article = main('stock market')
# print(len(cbc_tsx_article))
# print(cbc_tsx_article[0])

API CALLS SCRAPED:  20
ARTICLES IN URL LIST:  74
API CALLS SCRAPED:  40
ARTICLES IN URL LIST:  230
API CALLS SCRAPED:  60
ARTICLES IN URL LIST:  362
API CALLS SCRAPED:  80
ARTICLES IN URL LIST:  428
API CALLS SCRAPED:  100
ARTICLES IN URL LIST:  483
API CALLS SCRAPED:  120
ARTICLES IN URL LIST:  534
API CALLS SCRAPED:  140
ARTICLES IN URL LIST:  577
API CALLS SCRAPED:  160
ARTICLES IN URL LIST:  609
API CALLS SCRAPED:  180
ARTICLES IN URL LIST:  636
API CALLS SCRAPED:  200
ARTICLES IN URL LIST:  645
API CALLS SCRAPED:  220
ARTICLES IN URL LIST:  660
API CALLS SCRAPED:  240
ARTICLES IN URL LIST:  677
API CALLS SCRAPED:  260
ARTICLES IN URL LIST:  697
API CALLS SCRAPED:  280
ARTICLES IN URL LIST:  707
API CALLS SCRAPED:  300
ARTICLES IN URL LIST:  730
API CALLS SCRAPED:  320
ARTICLES IN URL LIST:  737
API CALLS SCRAPED:  340
ARTICLES IN URL LIST:  750
API CALLS SCRAPED:  360
ARTICLES IN URL LIST:  761
API CALLS SCRAPED:  380
ARTICLES IN URL LIST:  768
API CALLS SCRAPED:  400
ARTICLES IN 

KeyboardInterrupt: 

In [67]:
initial_sm_url = get_initial_url("stock market")
all_sm_urls = scrape_urls(initial_sm_url)

API CALLS SCRAPED:  20
ARTICLES IN URL LIST:  74
API CALLS SCRAPED:  40
ARTICLES IN URL LIST:  230
API CALLS SCRAPED:  60
ARTICLES IN URL LIST:  360
API CALLS SCRAPED:  80
ARTICLES IN URL LIST:  426
API CALLS SCRAPED:  100
ARTICLES IN URL LIST:  487
API CALLS SCRAPED:  120
ARTICLES IN URL LIST:  534
API CALLS SCRAPED:  140
ARTICLES IN URL LIST:  578
API CALLS SCRAPED:  160
ARTICLES IN URL LIST:  624
API CALLS SCRAPED:  180
ARTICLES IN URL LIST:  647
API CALLS SCRAPED:  200
ARTICLES IN URL LIST:  663
API CALLS SCRAPED:  220
ARTICLES IN URL LIST:  681
API CALLS SCRAPED:  240
ARTICLES IN URL LIST:  697
API CALLS SCRAPED:  260
ARTICLES IN URL LIST:  718
API CALLS SCRAPED:  280
ARTICLES IN URL LIST:  727
API CALLS SCRAPED:  300
ARTICLES IN URL LIST:  748
API CALLS SCRAPED:  320
ARTICLES IN URL LIST:  761
API CALLS SCRAPED:  340
ARTICLES IN URL LIST:  775
API CALLS SCRAPED:  360
ARTICLES IN URL LIST:  783
API CALLS SCRAPED:  380
ARTICLES IN URL LIST:  789
API CALLS SCRAPED:  400
ARTICLES IN 

In [71]:
json_list = []
json_count = 0
for each_url in all_sm_urls:
    retrieved_json  = extract_json_items(each_url)
    if retrieved_json is not None:
        #print(each_url)
        json_list.append(retrieved_json)

        json_count += 1
        if json_count % 20 == 0:
            print("JSONS ADDED:", json_count)

JSONS ADDED: 20
JSONS ADDED: 40
JSONS ADDED: 60
JSONS ADDED: 80
JSONS ADDED: 100
JSONS ADDED: 120
JSONS ADDED: 140
JSONS ADDED: 160
JSONS ADDED: 180
JSONS ADDED: 200
JSONS ADDED: 220
JSONS ADDED: 240
JSONS ADDED: 260
JSONS ADDED: 280
JSONS ADDED: 300
Error code:  404
JSONS ADDED: 320
JSONS ADDED: 340
JSONS ADDED: 360
Error code:  404
JSONS ADDED: 380
JSONS ADDED: 400
JSONS ADDED: 420
JSONS ADDED: 440
JSONS ADDED: 460
JSONS ADDED: 480
JSONS ADDED: 500
Error code:  404
JSONS ADDED: 520
JSONS ADDED: 540
JSONS ADDED: 560
JSONS ADDED: 580
JSONS ADDED: 600
JSONS ADDED: 620
JSONS ADDED: 640
JSONS ADDED: 660
JSONS ADDED: 680
JSONS ADDED: 700
JSONS ADDED: 720
JSONS ADDED: 740
JSONS ADDED: 760
JSONS ADDED: 780
JSONS ADDED: 800
JSONS ADDED: 820
JSONS ADDED: 840
JSONS ADDED: 860


In [73]:
query = "stock market"
full_query = query.split(" ")
file_name_prefix = "_".join(full_query)
print(file_name_prefix)

with open( file_name_prefix + '_' +'CBC_article' + '.json', 'w') as json_file:
    json.dump(json_list, json_file)

stock_market


In [ ]:
#TEST ON DIFFERENT CBC LINKS RETURNED BY CBC API

standard_url = "//www.cbc.ca/news/politics/federal-deficit-higher-than-252-billion-1.5566768"
radio_url = "//www.cbc.ca/radio/costofliving/slashed-interest-rates-getting-a-piece-of-the-electric-car-pie-and-a-happy-jobs-friday-to-all-1.5486253"
media_url = "//www.cbc.ca/player/play/1707317315674"
noauthor_url = "//www.cbc.ca/news/canada/coronavirus-covid19-world-canada-may12-1.5564261"



extract_json_items(standard_url)

#note: doesn't work for 'player' URLS ("//www.cbc.ca/player/play/1707317315674")
#will run, but will return null for most values - player articles have different setup
